#Kernel Non-negative Tensor Factorization

0. Loading dataset CMU Faces

In [1]:
import numpy as np
from sklearn.metrics import pairwise_kernels as K #to compute kernel matrix
import scipy.io as sio #to load mat files
from OKMF import OKMF #Online Kernel Matrix Factorization
from kntf import Kntf
import matplotlib.pyplot as plt

In [2]:
sio.whosmat('../datasets/CMU(30x11x21x1024).mat')

[('FullTensor', (30, 11, 21, 1024), 'uint8')]

In [3]:
matTF = sio.loadmat('../datasets/CMU(30x11x21x1024).mat')#loading tensor of formatted faces' images

In [4]:
TF = matTF['FullTensor']

In [9]:
TF.shape

(30, 11, 21, 1024)

1.1. Reshape tensor as matrix

In [10]:
MF = np.reshape(TF,(30*11*21,1024))

In [11]:
MF.shape

(6930, 1024)

In [21]:
one_face = np.asarray(np.reshape(TF[0,0,0,:],(32,32)))

In [22]:
one_face.shape

(32, 32)

In [23]:
imgplot = plt.imshow(one_face, cmap='Greys_r')

RuntimeError: Invalid DISPLAY variable

2. Computing Kernel Non-negative Tensor Factorization

2.1. Testing OKNMF (Online Kernel Non-negative Matrix Factorization) directly

Paramenters:

In [6]:
Gamma = 0.2
Lambda = 0.3
Alpha = 0.4
sigma = 2**1

In [16]:
ok = OKMF(1000,10,100,2,Gamma,Lambda,Alpha,'rbf',gamma=sigma)
#(budgetSize,latentTopics,minibatchSize,epochs,Gamma,Lambda,Alpha,metric,**kwds):

In [17]:
X = TF[:,0,0,:]

In [18]:
ok.fit(X,True)

In [10]:
sumErr = ok.trainErrors[0] + ok.trainErrors[1] + ok.trainErrors[2]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [150]:
estX = np.dot(ok.W,ok.H)

In [149]:
X.shape

(30, 1024)

In [168]:
estX.shape

(30, 30)

In [12]:
ok.trainErrors

array([], dtype=float64)

In [192]:
ok = OKMF(10000,10,100,2,Gamma,Lambda,Alpha,'precomputed',gamma=sigma)
#(budgetSize,latentTopics,minibatchSize,epochs,Gamma,Lambda,Alpha,metric,**kwds):

Precompute Kernel matrix

In [213]:
KF = K(MF,None,metric='linear')

In [214]:
KF = K(MF,None,'rbf',gamma=sigma)

In [215]:
ok.fit(KF,True)

2.1.1.2. Compute error

In [216]:
estKF = np.dot(ok.W,ok.H)

In [217]:
estKF.shape

(6930, 6930)

In [218]:
error = np.linalg.norm(estKF-KF,'fro')/np.linalg.norm(KF,'fro')

In [236]:
error

0.99989071251536932

2.1.1.3. Exploring parameters


In [243]:
eListxlt = []
latentTopic=5
for latentTopic in range(2,10):
    ok = OKMF(10000,latentTopic,100,10,Gamma,Lambda,Alpha,'precomputed',gamma=sigma)
    KF = K(MF,None,metric='linear')
    ok.fit(KF,True)
    eListxlt.append(np.linalg.norm(np.dot(ok.W,ok.H)-KF,'fro')/np.linalg.norm(KF,'fro'))
    


In [242]:
eListxlt

[0.99999999997385458]

In [ ]:
plt.plot(np.r_[2:5], eListxlt)
plt.show()

2.2. Adapting algorithm to tensors

In [170]:
KB = K(estX,None,metric='precomputed')

In [177]:
randX = np.random.rand(20,20,20,20)

In [193]:
KT = K(randX,None,metric='linear')

In [185]:
KT.shape

(20, 20, 20, 20, 20, 20)

# 3. Compute tensorial kernel

$\mathcal{X}\in\mathbb{R}_+^{30\times11\times21\times1024}$

In [8]:
np.shape(TF[:,0,0,:])

(30, 1024)

In [19]:
size_tensor = TF.shape

In [20]:
num_obj = size_tensor[0]*size_tensor[1]*size_tensor[2]
Ks = np.ones((num_obj,num_obj))
val_diff = 0.5
val_eq = 1
Ks = Ks*val_diff

#kernel for subjects (mode-0):
for i in range(size_tensor[0]):
    #compute Soft kernel for subjects
    Ks[i:(i+1)*(size_tensor[1]*size_tensor[2]),i:(i+1)*(size_tensor[1]*size_tensor[2])] = val_eq
    

In [9]:
MF.shape

(6930, 1024)

In [9]:
import kernel as Kernel
Ks = Kernel.softKernel(MF, 0.5, 1, 30)

In [10]:
Ki = Kernel.prodKernel(MF, MF[0,:],'rbf',gamma=sigma)

In [15]:
Ks[0,240:260]

array([ 0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5])

In [11]:
Ki.shape

(6930, 1)

In [31]:
MF = np.reshape(TF,(30*11*21,1024))

In [32]:
MF[231,:]-TF[1,0,0,:]

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [33]:
MF[0:21,:]-TF[0,0,:,:]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [14]:
#Kernel for poses
Kp = K(MF,None,'rbf',gamma=sigma)
#Kernel for illumination
Ki = K(MF,None,'linear')
Kr = np.dot(Ks, np.dot(Kp,Ki))

In [15]:
Kr.shape

(6930, 6930)

3.1 Computing kernel-NMF $\phi(X) \approx \phi(X)WH$

In [14]:
#online kernel NMF algorithm (by Esteban)
ok = OKMF(6930,10,100,2,Gamma,Lambda,Alpha,'rbf',gamma=sigma)
#(budgetSize,latentTopics,minibatchSize,epochs,Gamma,Lambda,Alpha,metric,**kwds):

In [23]:
TF[:29,:].shape

(29, 11, 21, 1024)

In [15]:
ok.fit(MF,True)

3.1.2 Parameters

In [13]:
Gamma = 0.2
Lambda = 0.3
Alpha = 0.4
sigma = 2**1

In [7]:
#My impelementation
ktf = Kntf(10,2,Gamma,Lambda,Alpha,'precomputed',gamma=sigma)


In [8]:
ktf.fit(TF,True)

kntf.py:103: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if Ks!=None:
kntf.py:198: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if indx!=None:
kntf.py:121: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if indx == None:
kntf.py:91: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if y==None:
kntf.py:112: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if Ks!=None:


ValueError: shapes (6930,) and (10,) not aligned: 6930 (dim 0) != 10 (dim 0)

In [13]:
int(ktf._T.shape[1]*ktf._T.shape[2]*(np.ceil(8/(ktf._T.shape[1]*ktf._T.shape[2]*1.0))-1))

0

In [14]:
int(ktf._T.shape[1]*ktf._T.shape[2]*(np.ceil(8/(ktf._T.shape[1]*ktf._T.shape[2]*1.0))))

231

In [10]:
Ks = ktf.SofKernelSubj(ktf._T,8)

In [13]:
Ks[231:240]

array([ 0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5])

3.2 Compute error as $||WH-\phi(X)||_F/||\phi(X)||_F$

In [20]:
error = np.linalg.norm(np.dot(ok.W,ok.H)-ok._KB,'fro')/np.linalg.norm(ok._KB,'fro')

In [19]:
error

0.9999999999728898

# Convex NMF approach

In [7]:
from convex_non_negative_matrix_factorization import CNMF

In [12]:
cnmf = CNMF()

(6930, 1024)